In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np 
import pandas as pd 
import nltk         
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import re 
from tqdm import tqdm,trange
import os   

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import json  
import pandas as pd
df = pd.read_csv('../../dataset/corpusfinal.csv')

In [6]:
df['article'] = df['document'].apply(lambda x: x.replace(". ",".\n"))
df['article'] = df['article'].apply(lambda x: x.split("\n"))
df["title"] = df["paper"]
df["summary"] = df["summary"]

In [7]:
def print_article(article):
    for item in article:
        print(item)
print("shit")
print_article( df['article'][0])

shit
The ninth International Conference on Forest Fire Research, together with the 17th International Wildland Fire Safety Summit was held in November 2022.
This joint conference brought together several hundred presentations in the field of wildfire research.
This special issue of the International Journal of Wildland Fire includes several papers describing work presented at the conference.
Due to the large number and variety of papers submitted, the special issue is being published in several parts.
Part 1 of the special issue was published, in January 2023, with a set of eight papers.
This Foreword to Part 2 of the special issue introduces 10 new papers on various topics that include: modelling of ignition probability, fire behaviour modelling, wildfire hazard mapping, fire management policies, imagery and mapping, and wildland–urban interface.
All papers in the special issue are published Open Access.


In [8]:
def sentence_tokenize(sent):
    lst = []
    tmp_lst = []
    for i in range(len(sent)):
        if not (len(tmp_lst)==0 and sent[i]==' '):
            tmp_lst.append(sent[i])
        if i==len(sent)-1 or (sent[i]=='.' and (not(sent[i+1].isdigit()) or sent[i+1]==" ")):
            lst.append(''.join(tmp_lst))
            tmp_lst = []
    return lst

In [9]:

def split_sent(article_list):
    sentences = []
    for s in article_list:
        sentences.extend(sentence_tokenize(s))  
    return sentences


In [10]:
sentences = split_sent(df['article'][0])
sentences

['The ninth International Conference on Forest Fire Research, together with the 17th International Wildland Fire Safety Summit was held in November 2022.',
 'This joint conference brought together several hundred presentations in the field of wildfire research.',
 'This special issue of the International Journal of Wildland Fire includes several papers describing work presented at the conference.',
 'Due to the large number and variety of papers submitted, the special issue is being published in several parts.',
 'Part 1 of the special issue was published, in January 2023, with a set of eight papers.',
 'This Foreword to Part 2 of the special issue introduces 10 new papers on various topics that include: modelling of ignition probability, fire behaviour modelling, wildfire hazard mapping, fire management policies, imagery and mapping, and wildland–urban interface.',
 'All papers in the special issue are published Open Access.']

In [11]:
print("length: ",len(sentences))

length:  7


In [12]:
# make alphabets lowercase
lower_sentences = [s.lower() for s in sentences]

In [13]:
print(sentences[0])
print(lower_sentences[0])

The ninth International Conference on Forest Fire Research, together with the 17th International Wildland Fire Safety Summit was held in November 2022.
the ninth international conference on forest fire research, together with the 17th international wildland fire safety summit was held in november 2022.


In [14]:
nltk.download('stopwords')# one time execution
from nltk.corpus import stopwords
print("stop word")
stop_words = stopwords.words('english')

stop word


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# 部分Stop word
stop_words[:30:2]

['i',
 'my',
 'we',
 'ours',
 'you',
 "you've",
 "you'd",
 'yours',
 'yourselves',
 'him',
 'himself',
 "she's",
 'hers',
 'it',
 'its']

In [16]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [17]:
# remove stopwords from the sentences
print("Stop word...")
clean_stopword_sentences = [remove_stopwords(r.split()) for r in lower_sentences]

Stop word...


In [18]:
print(lower_sentences[4])
print("-"*10,"Stop word","-"*10)
print(clean_stopword_sentences[4])

part 1 of the special issue was published, in january 2023, with a set of eight papers.
---------- Stop word ----------
part 1 special issue published, january 2023, set eight papers.


In [19]:
def clean_sentence(sentences):
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ",regex=True)
    return clean_sentences
clean_sentences = clean_sentence(clean_stopword_sentences)

In [20]:
print(clean_stopword_sentences[4])
print(clean_sentences[4])

part 1 special issue published, january 2023, set eight papers.
part   special issue published  january       set eight papers 


In [21]:
# Extract word vectors
def obtain_word_embeddings(path):
    word_embeddings = {}
    with open(path, encoding='utf-8') as f:
        for line in f:
            values = line.split()  # Split data
            word = values[0]       # Get its name
            coefs = np.asarray(values[1:], dtype='float32')  # Get its coefficient
            word_embeddings[word] = coefs
    return word_embeddings

In [22]:
dim = 100 # 50d, 100d, 200d, 300d
word_embeddings = obtain_word_embeddings(f'/content/drive/MyDrive/dataset/glove.6B.100d.txt')

In [23]:
print(len(word_embeddings['power']))
word_embeddings['power']

100


array([-4.0623e-02,  1.2911e-01,  9.2652e-01, -7.2253e-02,  4.3828e-01,
       -3.7762e-01, -2.7500e-01, -9.7944e-02, -1.7680e-01,  3.8279e-01,
        9.2663e-03,  9.0631e-03, -3.1502e-01,  6.2815e-02, -2.2111e-01,
       -9.9742e-01, -1.8360e-01,  3.9113e-01, -9.2952e-02, -1.2779e-01,
        4.8426e-01, -4.9320e-01,  5.0948e-01, -4.0813e-01,  6.3657e-01,
       -3.5722e-01, -2.9193e-01,  3.8334e-01,  5.3071e-01,  3.5986e-01,
        9.4441e-01,  5.1081e-01, -2.8931e-01, -1.8275e-01, -6.4469e-01,
        1.7839e-03,  2.9478e-01, -1.6024e-01, -3.6157e-01, -3.5547e-01,
       -1.7029e-01, -3.6866e-01,  2.1928e-01, -8.1945e-01, -9.6375e-02,
       -9.4109e-02,  3.1669e-01, -5.9285e-01,  5.9422e-01, -3.3568e-01,
       -5.5049e-01,  5.8094e-02, -2.0299e-02,  1.5526e+00,  1.0057e+00,
       -2.2807e+00,  9.0735e-02,  2.5548e-01,  1.9764e+00,  1.9240e-01,
        2.1717e-01, -5.1021e-01, -5.1359e-01,  5.1908e-01,  1.0555e+00,
        5.8991e-01,  3.1111e-01, -6.3756e-01,  8.6152e-02,  3.56

In [24]:
def get_sentence_vector(clean_sentences,word_embeddings,dim):
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0 and len(i.split())!=0:
            v = sum([word_embeddings.get(w, np.zeros((dim,))) for w in i.split()])  /  (len(i.split())) # 单词向量和 / 单词个数
        else:
            v = np.zeros((dim,))
        sentence_vectors.append(v)
    return sentence_vectors

In [25]:
sentence_vectors = get_sentence_vector(clean_sentences,word_embeddings,dim)

In [26]:
sentence_vectors[0]

array([-0.0866532 ,  0.08418871,  0.15094852,  0.05949325,  0.10869365,
       -0.08352906,  0.16700694,  0.2449452 , -0.37679264, -0.021509  ,
        0.10233872, -0.34138831,  0.26892778,  0.04602986, -0.11408847,
        0.03685861,  0.3067779 ,  0.03654407, -0.67514336, -0.315393  ,
        0.11380776, -0.1976444 ,  0.272058  ,  0.06429734,  0.07604373,
       -0.10811225, -0.2111848 , -0.01194093, -0.02708333,  0.06992132,
       -0.47495094,  0.097684  , -0.05942392,  0.35014325, -0.16982561,
        0.01340874, -0.07463146,  0.31187364, -0.07301001,  0.20158   ,
       -0.3868424 , -0.08900447,  0.15718105, -0.16183168,  0.3687393 ,
        0.04533866, -0.12615302, -0.08868994, -0.22620715, -0.4694387 ,
       -0.10418312, -0.30510396, -0.24356586,  0.6270906 , -0.0430106 ,
       -2.0217593 ,  0.05500619, -0.08854961,  1.467018  ,  0.39688653,
       -0.2677607 ,  0.46089813, -0.31362534, -0.10572854,  0.31463292,
        0.04667593, -0.107234  ,  0.20204054,  0.4642881 , -0.00

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
def get_sim_mat(sentences,sentence_vectors,dim):
    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,dim), sentence_vectors[j].reshape(1,dim))[0,0]
    return sim_mat

In [29]:
sim_mat = get_sim_mat(sentences,sentence_vectors,dim)

In [30]:
sim_mat[0]

array([0.        , 0.89824814, 0.8892498 , 0.78189045, 0.84094   ,
       0.75374377, 0.7802425 ])

In [31]:
!pip install networkx==2.6.3
import networkx as nx
def rank_sent(sim_mat,sentences):
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank_numpy(nx_graph)
    ranked_sentences = sorted(((scores[i],i) for i,s in enumerate(sentences)), reverse=True)
    return ranked_sentences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
ranked_sentences = rank_sent(sim_mat,sentences)

<ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(nx_graph)


In [33]:
ranked_sentences

[(0.15023195555017166, 2),
 (0.1461720618433313, 4),
 (0.14583468413423945, 3),
 (0.1428034499786075, 6),
 (0.14223010001595518, 1),
 (0.13902271195835617, 0),
 (0.13370503651933877, 5)]

In [34]:
def extract_summary(ranked_sentences, sentence_number):
    # Specify number of sentences to form the summary
    result_lst = []
    # Generate summary
    for i in range(sentence_number):
        result_lst.append(ranked_sentences[i][1])
    
    return result_lst

In [35]:
result_lst = extract_summary(ranked_sentences,5)

In [36]:
[sentences[i] for i in result_lst]

['This special issue of the International Journal of Wildland Fire includes several papers describing work presented at the conference.',
 'Part 1 of the special issue was published, in January 2023, with a set of eight papers.',
 'Due to the large number and variety of papers submitted, the special issue is being published in several parts.',
 'All papers in the special issue are published Open Access.',
 'This joint conference brought together several hundred presentations in the field of wildfire research.']

In [37]:
data = df['summary'][0]
data = sentence_tokenize(data)
gold = split_sent(data)

In [38]:
print(gold)

['The ninth International Conference on Forest Fire Research, together with the 17th International Wildland Fire Safety Summit was held in November 2022.', 'This joint conference brought together several hundred presentations in the field of wildfire research.', 'This special issue of the International Journal of Wildland Fire includes several papers describing work presented at the conference.', 'Due to the large number and variety of papers submitted, the special issue is being published in several parts.']


In [39]:
gold_lst = []
for i,sent in enumerate(sentences):
    if sent in gold:
        gold_lst.append(i)

In [40]:
result_lst

[2, 4, 3, 6, 1]

In [41]:
gold_lst

[0, 1, 2, 3]

In [42]:
correct = 0
for i in range(len(result_lst)):
    if result_lst[i] in gold_lst:
        correct+=1

In [43]:
correct

3

### Training data

In [44]:
pagerank = []
def test(i,word_embeddings,dim, start=0):
    sentences = split_sent(df['article'][start+i])
    stop_words = stopwords.words('english')
    lower_sentences = [s.lower() for s in sentences]
    clean_stopword_sentences = [remove_stopwords(r.split()) for r in lower_sentences]
    clean_sentences = clean_sentence(clean_stopword_sentences)
    sentence_vectors = get_sentence_vector(clean_sentences,word_embeddings,dim)
    sim_mat = get_sim_mat(sentences,sentence_vectors,dim)
    ranked_sentences = rank_sent(sim_mat,sentences)


    data = df['summary'][start+i]
    data = sentence_tokenize(data)
    gold = split_sent(data)
    sentence_num = len(gold)

    result_lst = extract_summary(ranked_sentences,sentence_num)
    pagerank.append(result_lst)
    result_lst.sort()
    gold_lst = []
    for i,sent in enumerate(sentences):
        if sent in gold:
            gold_lst.append(i)
    correct = 0
    for i in range(len(result_lst)):
        if result_lst[i] in gold_lst:
            correct+=1
    return correct/(sentence_num)

### Training

In [45]:
result = []
t = trange(1000)
dim = 100
word_embeddings = obtain_word_embeddings(f'/content/drive/MyDrive/dataset/glove.6B.100d.txt')

for i in t:
    accuracy = test(i,word_embeddings,dim)
    result.append(accuracy)
    t.set_postfix(Acc='%g' % accuracy)    
    

  0%|          | 0/1000 [00:00<?, ?it/s]<ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(nx_graph)
  0%|          | 1/1000 [00:21<5:54:54, 21.32s/it, Acc=0.5]<ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(nx_graph)
  0%|          | 2/1000 [00:21<2:27:34,  8.87s/it, Acc=0.333333]<ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(nx_graph)
  0%|          | 2/1000 [00:21<2:27:34,  8.87s/it, Acc=0.4]     <ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = n

In [46]:
sum(result)/len(result)

0.5145853174603175

In [47]:
data

['The ninth International Conference on Forest Fire Research, together with the 17th International Wildland Fire Safety Summit was held in November 2022.',
 'This joint conference brought together several hundred presentations in the field of wildfire research.',
 'This special issue of the International Journal of Wildland Fire includes several papers describing work presented at the conference.',
 'Due to the large number and variety of papers submitted, the special issue is being published in several parts.']

In [48]:
from rouge import Rouge
hypothesis = ''.join([sentences[i] for i in result_lst])
reference = ''.join([ i for i in data])
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)

In [49]:
scores

[{'rouge-1': {'r': 0.7647058823529411,
   'p': 0.7647058823529411,
   'f': 0.7647058773529413},
  'rouge-2': {'r': 0.676056338028169,
   'p': 0.676056338028169,
   'f': 0.676056333028169},
  'rouge-l': {'r': 0.7450980392156863,
   'p': 0.7450980392156863,
   'f': 0.7450980342156863}}]

In [50]:
import json
with open("/content/drive/MyDrive/dataset/training_scores.json","w") as f:
  json.dump(scores[0], f)

### Testing

In [51]:
result = []
t = trange(600)
dim = 100
word_embeddings = obtain_word_embeddings(f'/content/drive/MyDrive/dataset/glove.6B.100d.txt')

for i in t:
    accuracy = test(i,word_embeddings,dim,1001)
    result.append(accuracy)
    t.set_postfix(Acc='%g' % accuracy)    
    

  0%|          | 0/600 [00:00<?, ?it/s]<ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(nx_graph)
  0%|          | 1/600 [00:14<2:24:03, 14.43s/it, Acc=0.2]<ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(nx_graph)
  0%|          | 1/600 [00:14<2:24:03, 14.43s/it, Acc=0.5]<ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(nx_graph)
  0%|          | 1/600 [00:14<2:24:03, 14.43s/it, Acc=0.6]<ipython-input-31-70e71703fc72>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_num

In [52]:
from rouge import Rouge
hypothesis = ''.join([sentences[i] for i in result_lst])
reference = ''.join([ i for i in data])
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)

In [53]:
import json
with open("/content/drive/MyDrive/dataset/testing_scores.json","w") as f:
  json.dump(scores[0], f)